In [ ]:
import json

with open("json_data/classes_to_stories.json") as f:
    classes_to_stories = json.load(f)

batch_requests = []

for id, body in classes_to_stories.items():
    batch_requests.append({
        "custom_id": id,
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini-2024-07-18",
            "messages": [
                {
                    "role": "system", 
                    "content": f"You are a very objective assistant."},
                {
                    "role": "user",
                    "content": body["prompt"]
                }
            ],
            "max_tokens": 300,
            "temperature": 0.7
        }
    })

with open("json_data/batch_requests_class_to_story.jsonl", "w") as f:
    for obj in batch_requests:
        f.write(json.dumps(obj) + '\n')

In [24]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
OpenAI.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

batch_input_file = client.files.create(
    file=open("json_data/batch_requests_class_to_story.jsonl", "rb"),
    purpose="batch"
)

batch_input_file_id = batch_input_file.id

job_info = client.batches.create(
  input_file_id=batch_input_file_id,
  endpoint="/v1/chat/completions",
  completion_window="24h",
  metadata={
    "description": "story from class batch"
  }
)

print(job_info)

Batch(id='batch_676301f72f908190bcba5aa141167aaf', completion_window='24h', created_at=1734541815, endpoint='/v1/chat/completions', input_file_id='file-HAX1vFio5e9ADQeGnHHjeE', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1734628215, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'story from class batch'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [31]:
# copy from above
BATCH_ID = "batch_676301f72f908190bcba5aa141167aaf"

from openai import OpenAI
client = OpenAI()

print("Job Status:", client.batches.retrieve(BATCH_ID).status)

Job Status: completed


In [30]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
nltk.download('punkt_tab')

# negative is more negative, positive is more positive
def sentiment_analyzer(text):
    sid = SentimentIntensityAnalyzer()
    scores = []
    for sentence in nltk.sent_tokenize(text):
        scores.append(sid.polarity_scores(sentence)['compound'])
    return sum(scores) / len(scores)

print(sentiment_analyzer("She hates him."))

-0.4404


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\johns\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\johns\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [34]:
from openai import OpenAI
import json

client = OpenAI()

prompts = {}
with open("json_data/classes_to_stories.json", "r") as f:
    prompts = json.load(f)

file_response = client.files.content(client.batches.retrieve(BATCH_ID).output_file_id)

for line in file_response.text.split('\n'):
    if line:
        response = json.loads(line)
        req_id = response["custom_id"]
        prompts[req_id]["response"] = response["response"]["body"]["choices"][0]["message"]["content"]
        prompts[req_id]["sentiment"] = sentiment_analyzer(prompts[req_id]["response"])

with open("json_data/classes_to_stories.json", "w") as f:
    json.dump(prompts, f, indent=4)